In [6]:
import sagemaker
import boto3
import time
import json
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ SageMaker & Boto3 loaded!")


✅ SageMaker & Boto3 loaded!


In [7]:
# Set AWS Region & Account ID
aws_region = "ap-south-1"  # Change if needed
account_id = "125677696582"  # Your AWS Account ID

# Set ECR Image URI
image_uri = f"{account_id}.dkr.ecr.{aws_region}.amazonaws.com/tavi-extraction:latest"

# Define IAM Role for SageMaker
iam_role = "ArushSingh-SagemakerExecutionRole"  # Ensure SageMaker execution permissions

# Initialize SageMaker Session
sagemaker_session = sagemaker.Session(boto3.Session(region_name=aws_region))

print(f"✅ AWS Region: {aws_region}")
print(f"✅ Using Docker Image: {image_uri}")
print(f"✅ IAM Role: {iam_role}")


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


✅ AWS Region: ap-south-1
✅ Using Docker Image: 125677696582.dkr.ecr.ap-south-1.amazonaws.com/tavi-extraction:latest
✅ IAM Role: ArushSingh-SagemakerExecutionRole


In [8]:
# Generate unique model and endpoint names
model_name = f"tavivision-model-v2-1"
# endpoint_config_name = f"tavivision-endpoint-config"
endpoint_name = f"tavivision-endpoint-v2-1"

logger.info(f"Model Name: {model_name}")
# logger.info(f"Endpoint Config Name: {endpoint_config_name}")
logger.info(f"Endpoint Name: {endpoint_name}")


INFO:__main__:Model Name: tavivision-model-v2-1
INFO:__main__:Endpoint Name: tavivision-endpoint-v2-1


In [9]:
try:
    # Initialize the SageMaker Model
    model = sagemaker.model.Model(
        image_uri=image_uri,
        role=iam_role,
        sagemaker_session=sagemaker_session,
        name=model_name
    )
    
    logger.info("✅ Model initialization successful. Attempting deployment...")


except Exception as e:
    logger.error(f"❌ Deployment failed: {e}")
    raise


INFO:__main__:✅ Model initialization successful. Attempting deployment...


In [10]:
sm_client = boto3.client("sagemaker", region_name=aws_region)
iam = "arn:aws:iam::125677696582:role/ArushSingh-SagemakerExecutionRole"
# Create the model
sm_client.create_model(
    ModelName=model_name,
    PrimaryContainer={"Image": image_uri},
    ExecutionRoleArn=iam
)

{'ModelArn': 'arn:aws:sagemaker:ap-south-1:125677696582:model/tavivision-model-v2-1',
 'ResponseMetadata': {'RequestId': '1c9ebfc8-a36e-4752-aec3-ff18b821c530',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1c9ebfc8-a36e-4752-aec3-ff18b821c530',
   'strict-transport-security': 'max-age=47304000; includeSubDomains',
   'x-frame-options': 'DENY',
   'content-security-policy': "frame-ancestors 'none'",
   'cache-control': 'no-cache, no-store, must-revalidate',
   'x-content-type-options': 'nosniff',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '84',
   'date': 'Mon, 09 Feb 2026 10:26:42 GMT'},
  'RetryAttempts': 0}}

In [11]:
day_config = f"{endpoint_name}-config-day"
night_config = f"{endpoint_name}-config-night"
sm_client = boto3.client("sagemaker", region_name=aws_region)
# Day config
sm_client.create_endpoint_config(
    EndpointConfigName=day_config,
    ProductionVariants=[{
        "VariantName": "AllTraffic",
        "ModelName": model_name,
        "InstanceType": "ml.g4dn.2xlarge",  # big instance
        "InitialInstanceCount": 1
    }]
)

# Night config
sm_client.create_endpoint_config(
    EndpointConfigName=night_config,
    ProductionVariants=[{
        "VariantName": "AllTraffic",
        "ModelName": model_name,
        "InstanceType": "ml.g4dn.xlarge",  # smaller instance
        "InitialInstanceCount": 1
    }]
)

print(f"✅ Endpoint configs created: {day_config}, {night_config}")


✅ Endpoint configs created: tavivision-endpoint-v2-1-config-day, tavivision-endpoint-v2-1-config-night


In [12]:
sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=day_config  # Start with Day config
)

{'EndpointArn': 'arn:aws:sagemaker:ap-south-1:125677696582:endpoint/tavivision-endpoint-v2-1',
 'ResponseMetadata': {'RequestId': 'a274aea8-7ced-46b9-82a8-741318510785',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a274aea8-7ced-46b9-82a8-741318510785',
   'strict-transport-security': 'max-age=47304000; includeSubDomains',
   'x-frame-options': 'DENY',
   'content-security-policy': "frame-ancestors 'none'",
   'cache-control': 'no-cache, no-store, must-revalidate',
   'x-content-type-options': 'nosniff',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '93',
   'date': 'Mon, 09 Feb 2026 10:26:47 GMT'},
  'RetryAttempts': 0}}

In [13]:
import time

while True:
    response = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = response["EndpointStatus"]
    print(f"Endpoint status: {status}")
    
    if status in ["InService", "Failed"]:
        break
    time.sleep(30)  # check every 30s

Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint sta

In [9]:
import sagemaker
import boto3
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# AWS Region & Endpoint Name
aws_region = "ap-south-1"
endpoint_name = "tavivision-endpoint-v2-1"  # Replace with actual endpoint name

# Initialize SageMaker Session
sagemaker_session = sagemaker.Session()

# Check if endpoint already exists
client = boto3.client("sagemaker", region_name=aws_region)
response = client.describe_endpoint(EndpointName=endpoint_name)
status = response["EndpointStatus"]

if status == "InService":
    logger.info(f"✅ Endpoint {endpoint_name} is already running. No need to redeploy.")
else:
    # Deploy the model only if endpoint is NOT running
    model = sagemaker.model.Model(
        image_uri="242201296695.dkr.ecr.ap-southeast-1.amazonaws.com/tavivision",
        role="ArushSingh-SagemakerExecutionRole",  # Ensure SageMaker execution permissions
        sagemaker_session=sagemaker_session,
        name="neeraj-sagemaker-model"
    )

    predictor = model.deploy(
        initial_instance_count=1,
        instance_type="ml.g4dn.2xlarge",
        endpoint_name=endpoint_name
    )

    logger.info(f"✅ Model deployed! Endpoint: {predictor.endpoint_name}")


INFO:__main__:✅ Endpoint tavivision-endpoint-v2-1 is already running. No need to redeploy.


In [11]:
import boto3
import json

# AWS SageMaker details
aws_region = "ap-south-1"
endpoint_name = "tavivision-endpoint-v2-1"

# Initialize SageMaker runtime client
runtime_client = boto3.client("sagemaker-runtime", region_name=aws_region)

# Define input payload
payload = json.dumps({
    "task": "extract_pdf", 
    "pdf_url": "https://tavivision.s3.ap-south-1.amazonaws.com/pdfs/Bicuspid1a+report_ECG_PS49SE014_Landmark_Corelab+1.pdf"
})

# Invoke the SageMaker endpoint
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=payload
)

# Print the response
result = json.loads(response["Body"].read().decode())
print(json.dumps(result, indent=4))


{
    "execution_time": "28.41 seconds",
    "extracted_values": {
        "Annulus Area": "711.2",
        "Annulus Diameter": "30.1",
        "Annulus Perimeter": "96.8",
        "Annulus Perimeter Derived Diameter": "30.8",
        "Aortic Valve Anatomy Type": "Bicuspid Type 1a",
        "Asc Aorta Diameter": "42.2",
        "Calcium Score": "1917",
        "LCA Height": "15.6",
        "LVOT Diameter": "30.8",
        "RCA Height": "18.6",
        "SOV Height": "10.7",
        "SOV Left Diameter": "38.1",
        "SOV Non Diameter": "42.4",
        "SOV Right Diameter": "37.6",
        "STJ Diameter": "37.2",
        "url": "https://tavivision.s3.ap-south-1.amazonaws.com/TAVIVision/highlighted_pdf_report/5b16eabb-c86b-41f5-9438-3004b4dec759.pdf"
    },
    "femoral_values": {
        "CIA Left Diameter": 7.4,
        "CIA Right Diameter": 8.8,
        "EIA Left Diameter": 8.5,
        "EIA Right Diameter": 7.9,
        "FA Left Diameter": 7.6,
        "FA Right Diameter": 8.7,
    

In [ ]:
import boto3
import json

# AWS SageMaker details
aws_region = "ap-south-1"
endpoint_name = "tavivision-endpoint-v2"

# Initialize SageMaker runtime client
runtime_client = boto3.client("sagemaker-runtime", region_name=aws_region)

# Define input payload
payload = json.dumps({
    "task": "check-hardware"
})

# Invoke the SageMaker endpoint
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=payload
)

# Print the response
result1 = json.loads(response["Body"].read().decode())
print(json.dumps(result1, indent=4))


In [12]:
import boto3
import json

# AWS SageMaker details
aws_region = "ap-south-1"
endpoint_name = "tavivision-endpoint-v2-1"

# Initialize SageMaker runtime client
runtime_client = boto3.client("sagemaker-runtime", region_name=aws_region)

# Define input payload
payload = json.dumps({
    "task": "extract_pdf", 
    "pdf_url": "https://res.cloudinary.com/dkaa6ubzd/raw/upload/v1739864852/baojhvsbv3oxhioaidem"
})

# Invoke the SageMaker endpoint
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=payload
)

# Print the response
result = json.loads(response["Body"].read().decode())
print(json.dumps(result, indent=4))


{
    "execution_time": "29.44 seconds",
    "extracted_values": {
        "Annulus Area": "528.6",
        "Annulus Diameter": "25.9",
        "Annulus Perimeter": "81.9",
        "Annulus Perimeter Derived Diameter": "26.1",
        "Aortic Valve Anatomy Type": "Bicuspid Type 1a",
        "Asc Aorta Diameter": "43.0",
        "Calcium Score": "1449",
        "LCA Height": "21.6",
        "LVOT Diameter": "25.1",
        "RCA Height": "17.8",
        "SOV Height": "12.9",
        "SOV Left Diameter": "38.7",
        "SOV Non Diameter": "37.1",
        "SOV Right Diameter": "36.1",
        "STJ Diameter": "36.0",
        "url": "https://tavivision.s3.ap-south-1.amazonaws.com/TAVIVision/highlighted_pdf_report/95e05b08-307e-42eb-b31a-fce726d0ef13.pdf"
    },
    "femoral_values": {
        "CIA Left Diameter": 9.8,
        "CIA Right Diameter": 8.7,
        "EIA Left Diameter": 9.5,
        "EIA Right Diameter": 8.5,
        "FA Left Diameter": 8.6,
        "FA Right Diameter": 8.3,
    